In [ ]:
import glob
from datetime import datetime
from pandasql import sqldf
import pandas as pd
from ggplot import *
%matplotlib inline

pysqldf = lambda q: sqldf(q, globals())

In [ ]:
file_list = glob.glob('nypd-sqf-data/*.csv')
cols = ['datestop', 'race']

temp_df = pd.DataFrame()
for file in file_list:
#     print("Loading into dataframe: ", file)
    frame = pd.read_csv(file, usecols=cols, na_values=' ', low_memory=False)
    frame['total'] = len(frame)
    temp_df = pd.concat([temp_df, frame], ignore_index=True)


q = """
SELECT datestop, race, count(*) AS stops
FROM temp_df 
GROUP BY datestop, race
"""

df = pysqldf(q)
df.sample(5)

In [ ]:
ggplot(aes('race'), temp_df) + geom_histogram()

In [ ]:
out_df = df.loc[(df.race.str.contains("A|B|P|Q|W|Z")) & (df.race.notnull())].copy()

add_zero = lambda x: str(x).zfill(8)
out_df['datetime'] = pd.to_datetime(out_df['datestop'].map(add_zero), format="%m%d%Y", errors='coerce')
out_df.loc[out_df['datetime'].isnull(),'datetime'] = pd.to_datetime(out_df.loc[out_df['datetime'].isnull(),'datestop'])

In [ ]:
# Fix Dates
out_df = out_df.drop(['datestop'], 1)
out_df = out_df.dropna(subset=['datetime'])
out_df = out_df.loc[(out_df.datetime > '2000-01-01')]
out_df.head()

In [ ]:
# out_df.info()
ggplot(aes(x='datetime', y='stops', color='race'), out_df) + geom_line() 

In [ ]:
out_df['monthly'] = pd.to_datetime(out_df['datetime'].map(lambda x: str(x.year)+'/'+str(x.month)+'/01'))

In [ ]:
out_df.groupby(['monthly', 'race'])['stops'].sum().reset_index()

# # out_df = out_df.loc[(out_df.yearmonth > 200000)]

# out_df = out_df.dropna(subset=['yearmonth'])
# out_df = out_df.drop(['datetime'], 1)
# out_df = out_df.loc[(out_df.yearmonth > 200000)]

In [ ]:
out_df = out_df.drop(['datetime'], 1)
out_df = out_df.dropna(subset=['race'])

In [ ]:
tmp = out_df.groupby(['yearmonth', 'race'])['stops'].sum().reset_index()
tmp.head()

In [ ]:
tmp = tmp.loc[(tmp.yearmonth > '2000-01-01')]

In [ ]:
ggplot(aes(x='yearmonth', y='stops', color='race'), tmp) + geom_line()

In [ ]:

tmp = out_df.groupby(['yearmonth', 'race'])['stops'].sum().reset_index()
tmp['yearmonth'] = tmp['yearmonth'].map(lambda x: str(int(x))+'01')

tmp.head()

In [ ]:
for month in tmp.yearmonth.unique():
    tmp.loc[(tmp.yearmonth == month), 'mtotal'] = sum(tmp.loc[(tmp.yearmonth == month), 'stops'])

tmp.head()

In [ ]:
tmp['prop'] = tmp['stops']/tmp['mtotal']

tmp = tmp.drop(['stops', 'mtotal'], 1)
tmp.head()

In [ ]:
tmp.to_csv('test.csv')